In [28]:
def fetch(url):
    import requests, gzip, os, hashlib, numpy as np
    fp = os.path.join('tmp/' + hashlib.md5(url.encode('utf-8')).hexdigest() + '.gz')
    if os.path.isfile(fp):
        with open(fp, 'rb') as f:
            dat = f.read()
    else:
        with open(fp, 'wb') as f:
            dat = requests.get(url).content
            f.write(dat)
    return np.frombuffer(gzip.decompress(dat), dtype=np.uint8).copy()

X_train = fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")[16:].reshape((-1, 28, 28))
Y_train = fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")[8:]
X_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz")[16:].reshape((-1, 28, 28))
Y_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz")[8:]

In [2]:
import torch.nn as nn
import torch
class BobNet(nn.Module):
    def __init__(self):
        super(BobNet, self).__init__()
        self.l1 = nn.Linear(784, 128)
        self.act = nn.ReLU()
        self.l2 = nn.Linear(128, 10)
    def forward(self, x):
        x = self.l1(x)
        x = self.act(x)
        x = self.l2(x)
        return x


model = BobNet()

In [12]:
from tqdm import trange

In [13]:
batch = 32
optim = torch.optim.Adam(model.parameters())
for i in ( t := trange(10000)):
    samp = torch.randint(0, X_train.shape[0], (batch,))

    out = model(torch.tensor(X_train[samp]).flatten(1).float())

    loss = nn.functional.cross_entropy(out, torch.tensor(Y_train[samp]))

    optim.zero_grad()

    loss.backward()

    optim.step()

    t.set_description(f'loss {loss.item():.2f}')

loss 0.29:  10%|█         | 1006/10000 [00:01<00:13, 655.14it/s]


KeyboardInterrupt: 

In [27]:
Y_pred = torch.argmax(model(torch.tensor(X_test).flatten(1).float()), dim=1).numpy()

accuracy = (Y_pred == Y_test).mean()
accuracy

0.9395